In [1]:
!pip show sparseml-nightly

Name: sparseml-nightly
Version: 1.6.0.20231105
Summary: Libraries for applying sparsification recipes to neural networks with a few lines of code, enabling faster and smaller models
Home-page: https://github.com/neuralmagic/sparseml
Author: Neuralmagic, Inc.
Author-email: support@neuralmagic.com
License: Apache
Location: /home/rshaw/zephyr-training/sparseml-env/lib/python3.10/site-packages
Requires: click, GPUtil, ipywidgets, jupyter, matplotlib, merge-args, numpy, onnx, packaging, pandas, progressbar2, protobuf, psutil, pydantic, pyyaml, requests, scikit-image, scikit-learn, scipy, setuptools, sparsezoo-nightly, toposort, tqdm
Required-by: 


### **Confirm Model Is Working Okay**

In [2]:
# from transformers import AutoModelForCausalLM, AutoTokenizer

# model_id = "HuggingFaceH4/mistral-7b-sft-beta"
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForCausalLM.from_pretrained(model_id)

In [3]:
# from datasets import load_dataset

# dataset_id = "HuggingFaceH4/ultrachat_200k"
# dataset = load_dataset("HuggingFaceH4/ultrachat_200k")

In [4]:
# chat_template = tokenizer.apply_chat_template(dataset["train_sft"][0]["messages"][:-1], tokenize=False, add_generation_prompt=True)
# print(chat_template)

# tokens = tokenizer(chat_template, return_tensors="pt")

In [5]:
# preds = model.generate(**tokens, max_new_tokens=20)
# tokenizer.batch_decode(preds)

### **Construct Calib Dataset**

In [1]:
import tqdm
from sparseml.transformers.data.base_llm import TransformersDataset
from transformers import AutoTokenizer

system_prompt = {
    "content": "You are a friendly chatbot",
    "role": "system"
}

class ChatDataset(TransformersDataset):
    def __init__(
        self,
        model,
        seqlen,
        nsamples,
        path,
        seed: int = 0,
        split: str = "train_sft",
        split_percent_to_use: float = 1.0,
    ):
        super().__init__(
            model=model,
            seqlen=seqlen,
            nsamples=nsamples,
            path=path,
            name=None,
            seed=seed,
            split=split,
            use_max_tokens=False,
            split_percent_to_use=split_percent_to_use,
        )

        tok = AutoTokenizer.from_pretrained(model)

        processed_data = []
        for sample in tqdm.tqdm(self._data):
            assert "messages" in sample
            messages_with_sys_prompt = [system_prompt] + sample["messages"]
            processed_data.append(
                tok.apply_chat_template(
                    messages_with_sys_prompt, 
                    tokenize=False
                )
            )

        # print(processed_data[-1])
            
        self.create_dataloader(processed_data)

2023-11-06 17:58:34 sparseml.transformers WARNING  ****************************************************************
This is CRITICAL for the proper application of quantization in SparseML flows.

To resolve this, please run: `pip uninstall transformers;pip install nm-transformers`
Failing to do so is UNSUPPORTED and may significantly affect model performance.
****************************************************************


In [2]:
model_id = "HuggingFaceH4/mistral-7b-sft-beta"
dataset_id = "HuggingFaceH4/ultrachat_200k"

In [3]:
dataset = ChatDataset(
    model=model_id,
    seqlen=512,
    nsamples=512,
    path=dataset_id
)

calibration_data = dataset.loader
tokenizer = dataset.tokenizer

/home/rshaw/zephyr-training/sparseml-env/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)
100%|██████████| 512/512 [00:00<00:00, 12980.20it/s]


In [4]:
from transformers import AutoModelForCausalLM
import torch

model = AutoModelForCausalLM.from_pretrained(model_id)
print(model.dtype)
print(model.device)
print(model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

torch.float32
cpu
MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
   

In [5]:
import sparseml.core.session as session_manager
from sparseml.core.framework import Framework

recipe_file = "/home/rshaw/zephyr-training/pruning/recipe-50sparse.yaml"

session_manager.create_session()
session = session_manager.active_session()
session.apply(
    framework=Framework.pytorch,
    recipe=recipe_file,
    model=model,
    calib_data=calibration_data,
    start=0.0,
    device="cuda",
    copy_data=False,
)




------------------------------------------------------------
SKIPPING MODEL.to(device)
------------------------------------------------------------





2023-11-06 17:59:29 sparseml.modifiers.obcq.pytorch INFO     
===== Compressing layer 1/32 to sparsity 0.5 =====



----------------------------------------------------------------------------------------------
moving inputs to gpu...
----------------------------------------------------------------------------------------------
done...


2023-11-06 17:59:44 sparseml.modifiers.obcq.utils.layer_compressor INFO     Compressing module self_attn.q_proj of layer 0
2023-11-06 17:59:46 sparseml.modifiers.obcq.utils.sparsegpt INFO     time 1.31
2023-11-06 17:59:46 sparseml.modifiers.obcq.utils.sparsegpt INFO     error 0.05
2023-11-06 18:00:00 sparseml.modifiers.obcq.utils.layer_compressor INFO     Compressing module self_attn.k_proj of layer 0
2023-11-06 18:00:01 sparseml.modifiers.obcq.utils.sparsegpt INFO     time 1.24
2023-11-06 18:00:01 sparseml.modifiers.obcq.utils.sparsegpt INFO     error 0.12
2023-11-06 18:00:17 sparseml.modifiers.obcq.utils.layer_compressor INFO     Compressing module self_attn.v_proj of layer 0
2023-11-06 18:00:18 sparseml.modifiers.obcq.utils.sparsegpt INFO     time 1.26
2023-11-06 18:00:18 sparseml.modifiers.obcq.utils.sparsegpt INFO     error 0.30
2023-11-06 18:00:34 sparseml.modifiers.obcq.utils.layer_compressor INFO     Compressing module self_attn.o_proj of layer 0
2023-11-06 18:00:35 sparseml.mo

torch.cuda.memory_allocated: 4.457424GB
torch.cuda.max_memory_allocated: 7.816967GB
torch.cuda.max_memory_reserved: 8.798828GB



----------------------------------------------------------------------------------------------
moving inputs to gpu...
----------------------------------------------------------------------------------------------
done...


...

2023-11-06 19:19:41 sparseml.modifiers.obcq.utils.layer_compressor INFO     Compressing module self_attn.q_proj of layer 30
2023-11-06 19:19:42 sparseml.modifiers.obcq.utils.sparsegpt INFO     time 1.32
2023-11-06 19:19:42 sparseml.modifiers.obcq.utils.sparsegpt INFO     error 15666.87
2023-11-06 19:19:59 sparseml.modifiers.obcq.utils.layer_compressor INFO     Compressing module self_attn.k_proj of layer 30
2023-11-06 19:20:00 sparseml.modifiers.obcq.utils.sparsegpt INFO     time 1.32
2023-11-06 19:20:00 sparseml.modifiers.obcq.utils.sparsegpt INFO     error 4018.58
2023-11-06 19:20:17 sparseml.modifiers.obcq.utils.layer_compressor INFO     Compressing module self_attn.v_proj of layer 30
2023-11-06 19:20:19 sparseml.modifiers.obcq.utils.sparsegpt INFO     time 1.33
2023-11-06 19:20:19 sparseml.modifiers.obcq.utils.sparsegpt INFO     error 9264.97
2023-11-06 19:20:35 sparseml.modifiers.obcq.utils.layer_compressor INFO     Compressing module self_attn.o_proj of layer 30
2023-11-06 19:20:

torch.cuda.memory_allocated: 4.457302GB
torch.cuda.max_memory_allocated: 7.816967GB
torch.cuda.max_memory_reserved: 8.806641GB



----------------------------------------------------------------------------------------------
moving inputs to gpu...
----------------------------------------------------------------------------------------------
done...


2023-11-06 19:22:21 sparseml.modifiers.obcq.utils.layer_compressor INFO     Compressing module self_attn.q_proj of layer 31
2023-11-06 19:22:22 sparseml.modifiers.obcq.utils.sparsegpt INFO     time 1.32
2023-11-06 19:22:22 sparseml.modifiers.obcq.utils.sparsegpt INFO     error 14604.77
2023-11-06 19:22:39 sparseml.modifiers.obcq.utils.layer_compressor INFO     Compressing module self_attn.k_proj of layer 31
2023-11-06 19:22:41 sparseml.modifiers.obcq.utils.sparsegpt INFO     time 1.33
2023-11-06 19:22:41 sparseml.modifiers.obcq.utils.sparsegpt INFO     error 3725.28
2023-11-06 19:22:57 sparseml.modifiers.obcq.utils.layer_compressor INFO     Compressing module self_attn.v_proj of layer 31
2023-11-06 19:22:59 sparseml.modifiers.obcq.utils.sparsegpt INFO     time 1.34
2023-11-06 19:22:59 sparseml.modifiers.obcq.utils.sparsegpt INFO     error 9093.79
2023-11-06 19:23:15 sparseml.modifiers.obcq.utils.layer_compressor INFO     Compressing module self_attn.o_proj of layer 31
2023-11-06 19:23:

torch.cuda.memory_allocated: 4.457302GB
torch.cuda.max_memory_allocated: 7.816967GB
torch.cuda.max_memory_reserved: 8.806641GB




ModifiedState(model=MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
 

In [6]:
!pwd

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/rshaw/zephyr-training


In [7]:
save_path = "data/one-shot/50sparse-sft-v0"

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('data/one-shot/50sparse-sft-v0/tokenizer_config.json',
 'data/one-shot/50sparse-sft-v0/special_tokens_map.json',
 'data/one-shot/50sparse-sft-v0/tokenizer.json')

In [12]:
import os
from sparseml.optim.helpers import load_recipe_yaml_str

recipe_output_path = os.path.join(save_path, "recipe.yaml")
with open(recipe_output_path, "w") as fp:
    fp.write(load_recipe_yaml_str(recipe_file))

In [13]:
!ls {save_path}

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


config.json			  model-00006-of-00006.safetensors
generation_config.json		  model.safetensors.index.json
model-00001-of-00006.safetensors  recipe.yaml
model-00002-of-00006.safetensors  special_tokens_map.json
model-00003-of-00006.safetensors  tokenizer_config.json
model-00004-of-00006.safetensors  tokenizer.json
model-00005-of-00006.safetensors
